In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import os
import shutil

class VectorQuantizer(nn.Module):
    def __init__(self, num_embeddings, embedding_dim, commitment_cost):
        super(VectorQuantizer, self).__init__()
        self.embedding_dim = embedding_dim
        self.num_embeddings = num_embeddings
        self.commitment_cost = commitment_cost

        self.embedding = nn.Embedding(self.num_embeddings, self.embedding_dim)
        self.embedding.weight.data.uniform_(-1/self.num_embeddings, 1/self.num_embeddings)

    def forward(self, inputs):
        # Flatten input
        flat_input = inputs.view(-1, self.embedding_dim)

        distances = (torch.sum(flat_input**2, dim=1, keepdim=True) 
                    + torch.sum(self.embedding.weight**2, dim=1)
                    - 2 * torch.matmul(flat_input, self.embedding.weight.t()))
            
        encoding_indices = torch.argmin(distances, dim=1).unsqueeze(1)
        encodings = torch.zeros(encoding_indices.shape[0], self.num_embeddings, device=inputs.device)
        encodings.scatter_(1, encoding_indices, 1)
        
        quantized = torch.matmul(encodings, self.embedding.weight).view(inputs.shape)
        
        e_latent_loss = F.mse_loss(quantized.detach(), inputs)
        q_latent_loss = F.mse_loss(quantized, inputs.detach())
        loss = q_latent_loss + self.commitment_cost * e_latent_loss
        
        quantized = inputs + (quantized - inputs).detach()
        avg_probs = torch.mean(encodings, dim=0)
        perplexity = torch.exp(-torch.sum(avg_probs * torch.log(avg_probs + 1e-10)))
        
        return loss, quantized, perplexity, encodings

class Encoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, latent_dim):
        super(Encoder, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=2, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=2, padding=1)
        self.flatten = nn.Flatten()
        self.fc = nn.Linear(64 * 7 * 3, latent_dim)
 
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = self.flatten(x)
        z = self.fc(x)
        return z

class Decoder(nn.Module):
    def __init__(self, latent_dim, hidden_dim, output_dim):
        super(Decoder, self).__init__()
        self.fc = nn.Linear(latent_dim, 64 * 7 * 3)
        self.conv1 = nn.ConvTranspose2d(64, 32, kernel_size=3, stride=2, padding=1, output_padding=1)
        self.conv2 = nn.ConvTranspose2d(32, 1, kernel_size=3, stride=2, padding=(1,2), output_padding=(1,1))
 
    def forward(self, z):
        z = self.fc(z)
        z = z.view(-1, 64, 7, 3)
        z = F.relu(self.conv1(z))
        x_hat = torch.sigmoid(self.conv2(z))
        return x_hat

class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.conv1 = nn.Conv2d(1, 64, kernel_size=4, stride=2, padding=1)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1)
        self.conv3 = nn.Conv2d(128, 1, kernel_size=(4,2), stride=1, padding=2)
        
    def forward(self, x):
        x = F.leaky_relu(self.conv1(x), 0.2)
        x = F.leaky_relu(self.conv2(x), 0.2)
        x = torch.sigmoid(self.conv3(x))
        return x.view(-1, 1)

class VQVAEGAN(nn.Module):
    def __init__(self, encoder, decoder, vector_quantizer, discriminator):
        super(VQVAEGAN, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.vector_quantizer = vector_quantizer
        self.discriminator = discriminator
 
    def forward(self, x):
        z = self.encoder(x)
        loss, quantized, perplexity, _ = self.vector_quantizer(z)
        x_hat = self.decoder(quantized)
        return x_hat, loss, perplexity

# Update the loss function to include adversarial loss
def vqvae_gan_loss_f(x_hat, x, vq_loss, d_real, d_fake):
    recon_loss = F.mse_loss(x_hat, x, reduction='sum')
    gan_loss = F.binary_cross_entropy(d_fake, torch.ones_like(d_fake))
    total_loss = recon_loss + vq_loss + gan_loss
    return total_loss, recon_loss, vq_loss, gan_loss

def train_vqvae_gan(model, dataloader, optimizer_vqvae, optimizer_disc, criterion, device):
    model.train()
    running_loss = 0.0
    for batch_index, (x, _) in enumerate(dataloader):
        x = x.to(device)
        
        # Train Discriminator
        optimizer_disc.zero_grad()
        d_real = model.discriminator(x)
        recon_batch, vq_loss, _ = model(x)
        d_fake = model.discriminator(recon_batch.detach())
        
        disc_loss = F.binary_cross_entropy(d_real, torch.ones_like(d_real)) + \
                    F.binary_cross_entropy(d_fake, torch.zeros_like(d_fake))
        disc_loss.backward()
        optimizer_disc.step()
        
        # Train VQ-VAE
        optimizer_vqvae.zero_grad()
        recon_batch, vq_loss, _ = model(x)
        d_fake = model.discriminator(recon_batch)
        loss, _, _, _ = criterion(recon_batch, x, vq_loss, d_real, d_fake)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        running_loss += loss.item()
        optimizer_vqvae.step()
        
    return running_loss / len(dataloader.dataset)

def test_vqvae_gan(model, dataloader, criterion, device):
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for x, _ in dataloader:
            x = x.to(device)
            recon_batch, vq_loss, _ = model(x)
            d_real = model.discriminator(x)
            d_fake = model.discriminator(recon_batch)
            loss, _, _, _ = criterion(recon_batch, x, vq_loss, d_real, d_fake)
            test_loss += loss.item()
    return test_loss / len(dataloader.dataset)


# Define hyperparameters
latent_dim = 32
num_embeddings = 512
embedding_dim = 32
commitment_cost = 0.6
batch_size = 128
epochs = 5
save_dir = './result/checkpoint'
result_dir = './result/image'

# Create data loaders
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((28, 10), antialias=True)
])
train_dataset = datasets.MNIST('./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST('./data', train=False, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Create necessary directories
os.makedirs(save_dir, exist_ok=True)
os.makedirs(result_dir, exist_ok=True)

# Create VQ-VAE-GAN model
encoder = Encoder(1, 256, latent_dim)
decoder = Decoder(latent_dim, 256, 1)
vector_quantizer = VectorQuantizer(num_embeddings, embedding_dim, commitment_cost)
discriminator = Discriminator()
vqvae_gan = VQVAEGAN(encoder, decoder, vector_quantizer, discriminator)

# Define optimizers and loss function
optimizer_vqvae = optim.Adam(list(vqvae_gan.encoder.parameters()) + 
                             list(vqvae_gan.decoder.parameters()) + 
                             list(vqvae_gan.vector_quantizer.parameters()), lr=0.0001)
optimizer_disc = optim.Adam(vqvae_gan.discriminator.parameters(), lr=0.0001)
criterion = vqvae_gan_loss_f

# Learning rate scheduler
scheduler_vqvae = optim.lr_scheduler.StepLR(optimizer_vqvae, step_size=1, gamma=0.4)
scheduler_disc = optim.lr_scheduler.StepLR(optimizer_disc, step_size=1, gamma=0.4)

# Train VQ-VAE-GAN model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
vqvae_gan.to(device)
best_test_loss = float('inf')

def save_checkpoint(state, is_best, outdir):
    if not os.path.exists(outdir):
        os.makedirs(outdir)
    checkpoint_file = os.path.join(outdir, 'checkpoint.pth')
    best_file = os.path.join(outdir, 'model_best.pth')
    torch.save(state, checkpoint_file)
    if is_best:
        shutil.copyfile(checkpoint_file, best_file)
        

for epoch in range(epochs):
    train_loss = train_vqvae_gan(vqvae_gan, train_loader, optimizer_vqvae, optimizer_disc, criterion, device)
    test_loss = test_vqvae_gan(vqvae_gan, test_loader, criterion, device)
    scheduler_vqvae.step()
    scheduler_disc.step()
    print(f"Epoch {epoch+1}/{epochs}, Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}")
    
    # Save checkpoint
    is_best = test_loss < best_test_loss
    best_test_loss = min(test_loss, best_test_loss)
    save_checkpoint({
        'epoch': epoch,
        'state_dict': vqvae_gan.state_dict(),
        'best_test_loss': best_test_loss,
        'optimizer_vqvae': optimizer_vqvae.state_dict(),
        'optimizer_disc': optimizer_disc.state_dict(),
    }, is_best, save_dir)

# Generate fake 28x10 images
def generate_fake_images(model, num_images, device):
    model.eval()
    with torch.no_grad():
        z = torch.randn(num_images, latent_dim).to(device)
        fake_images = model.decoder(z)
    return fake_images

# Generate and display fake images
num_fake_images = 16
fake_images = generate_fake_images(vqvae_gan, num_fake_images, device)

# Display fake images
fig, axes = plt.subplots(4, 4, figsize=(15, 15))
for i, ax in enumerate(axes.flatten()):
    ax.imshow(fake_images[i].cpu().squeeze(), cmap='gray')
    ax.axis('off')
plt.tight_layout()
plt.savefig(os.path.join(result_dir, 'vqvae_gan_fake_images.png'))
plt.close()

print(f"Fake images saved to {os.path.join(result_dir, 'vqvae_gan_fake_images.png')}")

# Optional: Display the saved image
from PIL import Image
saved_image = Image.open(os.path.join(result_dir, 'vqvae_gan_fake_images.png'))
plt.figure(figsize=(20, 20))
plt.imshow(saved_image)
plt.axis('off')
plt.show()

C:\Users\manju\anaconda3\Lib\site-packages\torch\nn\modules\conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ..\aten\src\ATen\native\cudnn\Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,
C:\Users\manju\anaconda3\Lib\site-packages\torch\autograd\graph.py:744: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ..\aten\src\ATen\native\cudnn\Conv_v8.cpp:919.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


Epoch 1/5, Train Loss: 85.1610, Test Loss: 266.8795
Epoch 2/5, Train Loss: 561.7860, Test Loss: 1045.8952
Epoch 3/5, Train Loss: 1312.7126, Test Loss: 1711.3798


KeyboardInterrupt: 